In [1]:
import casadi as cs
from urdf_parser_py.urdf import URDF, Pose
import os # For current directory
import urdf2casadi.urdf2casadi.urdfparser
import urdf2casadi.urdf2casadi.geometry.plucker as plucker
import numpy as np
import PyKDL as kdl
import kdl_parser.kdl_parser_py.kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl

In [2]:
ok, ur_tree = kdlurdf.treeFromFile('./urdf2casadi/examples/urdf/test.urdf')
#ok, ur_tree = kdlurdf.treeFromFile('pantilt.urdf')
asd = urdf2casadi.urdf2casadi.urdfparser.URDFparser()
robot_desc = asd.from_file("./urdf2casadi/examples/urdf/test.urdf")
#robot_desc = asd.from_file("pantilt.urdf")
root = 'base_link'
tip = 'wrist_3_link5'
ur_chain = ur_tree.getChain(root,tip)
rbdlmodel = rbdl.loadModel("./urdf2casadi/examples/urdf/test.urdf")

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
./urdf2casadi/examples/urdf/test.urdf


In [3]:
jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = len(jlist)
print njoints
print len(qmax)

H_ur = kdl.JntSpaceInertiaMatrix(6)

grav = kdl.Vector()
grav[2] = -9.81

q = kdl.JntArray(njoints)
qdot = kdl.JntArray(njoints)

q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)

q_sx = cs.SX.zeros(njoints)
print q_sx
qdot_sx = cs.SX.zeros(njoints)

q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints

tau_kdl = kdl.JntArray(njoints)
grav_kdl = kdl.JntArray(njoints)

nitr = 50000
nrepeat = 3

M_rbdl = (rbdlmodel.q_size, rbdlmodel.q_size)
M_rbdl = np.zeros(M_rbdl)


30
30
@1=0, [@1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1]


# Forward Dynamics CRBA + RNEA vs ABA Timing
## u2c:

In [4]:
tau_u2c = [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]
fd_crba = asd.get_forward_dynamics_CRBA(root, tip, tau_u2c)
fd_aba = asd.get_forward_dynamics_ABA(root, tip, tau_u2c)

In [5]:
def fdcrba_u2c():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    fd_crba(q_none, qdot_none)

timeit_fdcrba = timeit("fdcrba_u2c()", setup = "from __main__ import fdcrba_u2c", number = 50000)
print timeit_fdcrba
print timeit_fdcrba/50000
timeit_fdcrba_repeat = repeat("fdcrba_u2c()", setup = "from __main__ import fdcrba_u2c", repeat = 8, number = 50000)
print timeit_fdcrba_repeat

10.6183600426
0.000212367200851
[11.98826289176941, 11.22599482536316, 17.93891978263855, 12.036670923233032, 12.208286046981812, 21.647587060928345, 11.580895185470581, 11.470958948135376]


In [6]:
def fdaba_u2c():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    fd_aba(q_none, qdot_none)

timeit_fdaba = timeit("fdaba_u2c()", setup = "from __main__ import fdaba_u2c", number = 50000)
print timeit_fdaba
print timeit_fdaba/50000
timeit_fdaba_repeat = repeat("fdaba_u2c()", setup = "from __main__ import fdaba_u2c", repeat = 8, number = 50000)
print timeit_fdaba_repeat

5.50214791298
0.00011004295826
[5.3047990798950195, 5.2741639614105225, 5.230039119720459, 5.273496150970459, 5.454323053359985, 5.6985859870910645, 5.222710847854614, 5.056975841522217]


In [7]:
tau_rbdl = np.ones(njoints)

In [8]:
def fd_rbdl():   
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.ForwardDynamics(rbdlmodel, q_np, qdot_np, tau_rbdl, qddot_np)

timeit_fdrbdl_repeat = repeat("fd_rbdl()", setup = "from __main__ import fd_rbdl", repeat = 8, number = 50000)
print timeit_fdrbdl_repeat

[4.847084045410156, 4.648300886154175, 4.7603349685668945, 4.7387731075286865, 4.753636837005615, 4.699927091598511, 4.737782955169678, 4.683664798736572]


## Inverse Dynamics Timing

In [9]:
id_tau = asd.get_inverse_dynamics_RNEA(root, tip, grav)
%%timeit 
for j in range(njoints):
    q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    qddot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2


id_tau(q_np, qdot_np, qddot_np)

SyntaxError: invalid syntax (<ipython-input-9-0b7eafda3f24>, line 2)